<a href="https://colab.research.google.com/github/BrianKEverett/County-Line/blob/main/PS6_Everett.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1537]:
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import missingno as msno

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )
data_table.max_columns = 50
#!and may have to click at top right spreadsheet icon

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

In [1538]:
permits=pd.read_csv("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Permits.csv")
# dataset can be found here: https://njdca.maps.arcgis.com/home/item.html?id=c754e8f800424bcbb6ad4e6e85b9f736 from NJ Dept of Community Affairs Website.
#this dataset was chosen mostly to assist with my dissertation proposal - I am seeking to explore the behaviors of planning boards in New Jersey, and one hypothesis I have is that planning board decisions are influenced by the effects of the County Line Balloting system, which is unique to 19 out of 21 counties in New Jersey. No other state in the US runs primary elections in this way.
#More on the County Line can be found here by Julia Sass Rubin: https://www.njpp.org/wp-content/uploads/2021/01/NJPP-Report-Does-the-County-Line-Matter-Update-wiht-Final-Vote-Counts.pdf

taxes=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/mediantax.csv")
# dataset can be retireved via: https://njdca.maps.arcgis.com/apps/webappviewer/index.html?id=96ec274c50a34890b23263f101e4ad9b from NJ Department of Community Affairs
# Another hypothesis I have is that the public narrative put forth by planning board members when approving controversial permits, "this will increase rateables for the township, lowering your taxes", does not actually come to fruition
#This data set is helpful for exploring that narrative, and seeing if the opposite is occurring, i.e. more development actually yields higher property taxes

#health=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/countyhealth.csv")
#dataset can be found here: https://www.countyhealthrankings.org/explore-health-rankings/rankings-data-documentation from County Health Rankings and Roadmaps, for 2022 to match same year of data for permits dataset
#With County Health Data, we can infer hypotheses about rates of development and the effect on well-being, liveability.

#Problem with health data set == only 22 observations for New Jersey, not a good sample.

municodes=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Municodes.csv")
#Data file of all NJ municiaplities, and counties, with the corresponding municipalitiy DCA code. This data file will be most helpful for matching and merging.

#crime=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/CamdenCrime.csv") #not a good format for reading data!
#Dataset can be retireved here: https://www.nj.gov/njsp/ucr/uniform-crime-reports.shtml on the NJ Office of the Attorney General's website
#This crime data is important to consider when analyzing planning and zoning. Does any specific type of development correlate with increased crime? Can liveability theory be worked in here for whether or not communities have what they need to prevent crime?

jobs=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/jobsdensity.csv')
#dataset can be built via the table selections on the NJ Community Affairs website - https://njdca.maps.arcgis.com/apps/webappviewer/index.html?id=96ec274c50a34890b23263f101e4ad9b
#Job density is a good variable to consider regarding new large dollar permits. Are some places growing more than others? Can this be attribute to the phenomenon of the County Line?

In [1539]:
del permits['ID']
del permits['BLOCK NUMBER']
del permits['PAMS PIN']
del permits['USE GROUP']
del permits['YCOORD']
del permits['XCOORD']
del permits['MATCH TYPE']
del permits['LOT NUMBER']
del permits['DATE ISSUED']

permits = permits.rename(columns={'DCA MUNI CODE': 'DCA'})
permits = permits.rename(columns={'MUNICIPALITY': 'Municipality'})

del municodes['MUNICIPALITY_NAME_NJ-1040']
del municodes['MUNICIPALITY_CODE_DCA']
del municodes['MUNICIPALITY_NAME_DCA']
del municodes['MUNICIPALITY_CODE_GNIS']
del municodes['MUNICIPALITY_NAME_GNIS']
del municodes['MUNICIPALITY_CODE_FIPS']
municodes = municodes.rename(columns={'MUNICIPALITY_NAME_COMMON': 'Municipality'})
municodes = municodes.rename(columns={'MUNICIPALITY_CODE_NJ-1040': 'DCA'})
municodes = municodes.rename(columns={'COUNTY_NAME_COMMON': 'County'})

del jobs['Blk_Grp_Name']
del jobs['JobsVintage']

del taxes['Data_Vintage']
del taxes['Tract_Name']

In [1540]:
jobs

,Municipality,County,Jobs,JobsDensity
0,West Caldwell Township,Essex,4376,4219.922635
1,West Caldwell Township,Essex,28,85.385941
2,West Caldwell Township,Essex,1939,1640.631286
3,Monroe Township,Gloucester,226,589.619454
4,West Deptford Township,Gloucester,156,258.364616
...,...,...,...,...
1995,Marlboro Township,Monmouth,90,107.930966
1996,Marlboro Township,Monmouth,243,233.244240
1997,Marlboro Township,Monmouth,2738,745.848682
1998,Dumont Borough,Bergen,10,80.075270


In [1541]:
municodes

,County,Municipality,DCA
0,Atlantic County,Absecon,101
1,Atlantic County,Atlantic City,102
2,Atlantic County,Brigantine,103
3,Atlantic County,Buena Borough,104
4,Atlantic County,Buena Vista Township,105
...,...,...,...
559,Warren County,Phillipsburg,2119
560,Warren County,Pohatcong Township,2120
561,Warren County,Washington Borough,2121
562,Warren County,Washington Township,2122


In [1542]:
munijobs = pd.merge(municodes, jobs, how='inner', on=['Municipality'],indicator=True)

In [1543]:
munijobs._merge.value_counts()

both          1747
left_only        0
right_only       0
Name: _merge, dtype: int64

In [1544]:
munijobs

,County_x,Municipality,DCA,County_y,Jobs,JobsDensity,_merge
0,Atlantic County,Hamilton Township,112,Mercer,1760,2515.536978,both
1,Atlantic County,Hamilton Township,112,Mercer,48,131.570863,both
2,Atlantic County,Hamilton Township,112,Mercer,546,3306.602757,both
3,Atlantic County,Hamilton Township,112,Mercer,268,373.710382,both
4,Atlantic County,Hamilton Township,112,Mercer,489,89.430318,both
...,...,...,...,...,...,...,...
1742,Warren County,Liberty Township,2114,Warren,346,36.054178,both
1743,Warren County,Liberty Township,2114,Warren,29,14.008905,both
1744,Warren County,White Township,2123,Warren,1145,101.858716,both
1745,Warren County,White Township,2123,Warren,327,98.444193,both


In [1545]:
munijobs = munijobs.rename(columns={'County_x': 'County'})

In [1546]:
munijobs

,County,Municipality,DCA,County_y,Jobs,JobsDensity,_merge
0,Atlantic County,Hamilton Township,112,Mercer,1760,2515.536978,both
1,Atlantic County,Hamilton Township,112,Mercer,48,131.570863,both
2,Atlantic County,Hamilton Township,112,Mercer,546,3306.602757,both
3,Atlantic County,Hamilton Township,112,Mercer,268,373.710382,both
4,Atlantic County,Hamilton Township,112,Mercer,489,89.430318,both
...,...,...,...,...,...,...,...
1742,Warren County,Liberty Township,2114,Warren,346,36.054178,both
1743,Warren County,Liberty Township,2114,Warren,29,14.008905,both
1744,Warren County,White Township,2123,Warren,1145,101.858716,both
1745,Warren County,White Township,2123,Warren,327,98.444193,both


In [1554]:
taxes

,County,Municipality,Median_RE_Taxes,
0,Somerset,Bridgewater Township,10000.0,
1,Somerset,Bernards Township,10000.0,
2,Somerset,Franklin Township,9604.0,
3,Middlesex,Woodbridge Township,9041.0,
4,Middlesex,Woodbridge Township,7496.0,
...,...,...,...,...
1641,Somerset,Franklin Township,10000.0,
1642,Somerset,Franklin Township,6600.0,
1643,Somerset,Franklin Township,10000.0,
1644,Somerset,Bernards Township,10000.0,


In [1547]:
del munijobs['_merge']
new = pd.merge(munijobs, taxes, how='inner', on=['County'],indicator=True)

In [1548]:
new._merge.value_counts()

left_only     0
right_only    0
both          0
Name: _merge, dtype: int64

In [1549]:
new

,County,Municipality_x,DCA,County_y,Jobs,JobsDensity,Municipality_y,Median_RE_Taxes,,_merge


In [1550]:
final = pd.merge(permits, new, how='inner', on=['DCA'],indicator='exists')

In [1551]:
final._merge.value_counts()

left_only     0
right_only    0
both          0
Name: _merge, dtype: int64

In [1552]:
final['DCA'][final['_merge']=='left_only']

Series([], Name: DCA, dtype: int64)

In [1553]:
final

,TAX CODE,DCA,Municipality,Use Group Label,TYPE,WORK VALUE,County,Municipality_x,County_y,Jobs,JobsDensity,Municipality_y,Median_RE_Taxes,,_merge,exists
